**reference--https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/**

- To manage memory run this code in terminal - -- jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000
- Import libraries

In [1]:
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp

# Load a CSV file
- Let's extract data from csv
- Change strings to float
- Chnage strings columns to integers
- Normalize dataset

In [2]:
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset
def str_column_to_float(datset, column):
    for row in dataset:
        row[column] = float(row[column].strip())
        
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

def dataset_minmax(dataset):
    minmax = list()
    stats = [[min(column), max(column)] for column in zip(*dataset)]
    return stats

def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)-1):
            if(minmax[i][1]-minmax[i][0] !=0):
                row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])
            else: row[i] = row[i]=0;

# Test training backprop algorithm in seed data

- Use of cross validation
- Split train and test set
- Split a dataset into k folds**


In [3]:
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Calculate accuracy percentage

In [4]:
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0


**Evaluate an algorithm using a cross validation split**

**input	 dataset)train patterns**

**input	 (algorithm)function to init and train the network + perform predictions**

**input	 (n_folds)dataset spliter number**

**input	 (*args) = (l_rate, n_epoch, n_hidden)**

In [5]:
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

In [6]:
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation


In [7]:
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

In [8]:
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs


# def forward_propagate(network, row):
#     inputs = row
#     #hidden layer
#     layer = network[0]
#     new_inputs = []
#     for neuron in layer:
#         activation = activate(neuron['weights'], inputs)
#         neuron['output'] = transfer(activation)
#         new_inputs.append(neuron['output'])
#         inputs = new_inputs
#         #output layer
#         layer = network[1]
#         new_inputs = []
#         for neuron in layer:
#             activation = activate(neuron['weights'], inputs)
#             neuron['output'] = softmax(activation)
#             new_inputs.append(neuron['output'])
#         inputs = new_inputs
#     return inputs

In [9]:
def transfer_derivative(output):
    return output * (1.0 - output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [10]:
# Update network weights with error
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']


In [11]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
    #accuracy = []
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))


# Initialize a network

Below is a function named initialize_network() that creates a new neural network ready for training. It accepts three parameters, the number of inputs, the number of neurons to have in the hidden layer and the number of outputs.


We can see that for the hidden layer we create n_hidden neurons and each neuron in the hidden layer has n_inputs + 1 weights, one for each input column in a dataset and an additional one for the bias.

We can also see that the output layer that connects to the hidden layer has n_outputs neurons, each with n_hidden + 1 weights. This means that each neuron in the output layer connects to (has a weight for) each neuron in the hidden layer.

- Create dictionary to store WEIGHTS as: [{'weights': [x x x] }, {...}, {'weights': [x x x]}]

In [12]:
# Initialize a network
# def initialize_network(n_inputs, n_hidden, n_outputs):
#     network = list()
#     hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
#     network.append(hidden_layer)
#     output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
#     network.append(output_layer)
#     return network


# def initialize_network(n_inputs, hidden_list, n_outputs):
#     network = list()
#     hidden_layer = [{'weights': [random() for i in range(n_inputs + 1)]} for i in range(hidden_list[0])]
#     network.append(hidden_layer)
#     for h in range(len(hidden_list) - 1):
#         # print("h:",h)
#         hidden_layer = [{'weights': [random() for i in range(hidden_list[h] + 1)]} for i in range(hidden_list[h + 1])]
#         network.append(hidden_layer)

#     output_layer = [{'weights': [random() for i in range(hidden_list[-1] + 1)]} for i in range(n_outputs)]
#     network.append(output_layer)
#     for layer in network:
#         print("\nlayer no:", network.index(layer))
#         for neuron in layer:
#             print("neuron no:", layer.index(neuron))
#             print(neuron)
#     return network

def initialize_network(n_inputs, hidden_list, n_outputs):
    network = list()
    hidden_layer = [{'weights': [random() for i in range(n_inputs + 1)]} for i in range(hidden_list[0])]
    network.append(hidden_layer)
    for h in range(len(hidden_list) - 1):
        # print("h:",h)
        hidden_layer = [{'weights': [random() for i in range(hidden_list[h] + 1)]} for i in range(hidden_list[h + 1])]
        network.append(hidden_layer)

    output_layer = [{'weights': [random() for i in range(hidden_list[-1] + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    for layer in network:
        print("\nlayer no:", network.index(layer))
        for neuron in layer:
            print("neuron no:", layer.index(neuron))
            print(neuron)
    return network


In [13]:
# Make a prediction with a network
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

In [14]:
# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
    n_inputs = len(train[0]) - 1
    n_outputs = len(set([row[-1] for row in train]))
    network = initialize_network(n_inputs, n_hidden, n_outputs)
    train_network(network, train, l_rate, n_epoch, n_outputs)
    predictions = list()
    for row in test:
        prediction = predict(network, row)
        predictions.append(prediction)
    return(predictions)

In [15]:
seed(1)

# load and prepare data
#filename = './data/only_position.csv'
filename = './data/train.csv'
dataset = load_csv(filename)[1:]

In [16]:
#dataset

In [17]:
for i in range(len(dataset[1])-1):
    str_column_to_float(dataset, i)
str_column_to_int(dataset, len(dataset[0])-1) # this column contain the grain type
# normalize input variables
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)

In [18]:
#minmax

In [19]:
len(dataset[0])-1

62

# evaluate algorithm

In [20]:
n_folds =10  # devide the data set in 10 sub set that are going to be used as training one by one
l_rate = 0.8 # number of iteration for the training
#n_hidden = 10
hidden_list = [10, 7, 3]
n_epoch = 200

scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, hidden_list)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))



layer no: 0
neuron no: 0
{'weights': [0.5238954398669781, 0.030700276754036526, 0.6749025775182691, 0.8033855134341553, 0.6598238917510797, 0.4262993787138545, 0.7374512500957098, 0.12568332230972723, 0.21213169303189394, 0.04744017352451846, 0.07072687788690923, 0.07644615553590373, 0.9171763132459553, 0.29787980574251793, 0.15820738983282634, 0.5649407226767994, 0.13039112842440792, 0.5607173210286936, 0.850526660963271, 0.5905839712874152, 0.21759033920020698, 0.9008129952123803, 0.460852490153291, 0.8279131566567798, 0.8698864279223623, 0.7800172694327171, 0.6229628138905298, 0.03742337508476401, 0.20040745546617267, 0.0990253627729546, 0.5733827030223086, 0.8965657460164287, 0.5914093121448057, 0.4923507504494802, 0.9379539017823154, 0.3900605387693097, 0.5041073672458524, 0.01720019672323503, 0.6121283611030989, 0.40232471301911965, 0.2813517154611124, 0.15696529448667917, 0.8575367430429213, 0.8111390619505175, 0.5633405356310336, 0.1351431409947249, 0.429240414597316, 0.266535

>epoch=6, lrate=0.800, error=100.256
>epoch=7, lrate=0.800, error=100.193
>epoch=8, lrate=0.800, error=100.049
>epoch=9, lrate=0.800, error=99.618
>epoch=10, lrate=0.800, error=98.954
>epoch=11, lrate=0.800, error=98.700
>epoch=12, lrate=0.800, error=98.648
>epoch=13, lrate=0.800, error=98.625
>epoch=14, lrate=0.800, error=98.610
>epoch=15, lrate=0.800, error=98.595
>epoch=16, lrate=0.800, error=98.577
>epoch=17, lrate=0.800, error=98.551
>epoch=18, lrate=0.800, error=98.504
>epoch=19, lrate=0.800, error=98.389
>epoch=20, lrate=0.800, error=97.978
>epoch=21, lrate=0.800, error=97.218
>epoch=22, lrate=0.800, error=96.945
>epoch=23, lrate=0.800, error=96.890
>epoch=24, lrate=0.800, error=96.871
>epoch=25, lrate=0.800, error=96.862
>epoch=26, lrate=0.800, error=96.856
>epoch=27, lrate=0.800, error=96.852
>epoch=28, lrate=0.800, error=96.850
>epoch=29, lrate=0.800, error=96.848
>epoch=30, lrate=0.800, error=96.846
>epoch=31, lrate=0.800, error=96.845
>epoch=32, lrate=0.800, error=96.844
>e

>epoch=2, lrate=0.800, error=101.388
>epoch=3, lrate=0.800, error=100.297
>epoch=4, lrate=0.800, error=99.013
>epoch=5, lrate=0.800, error=98.594
>epoch=6, lrate=0.800, error=98.404
>epoch=7, lrate=0.800, error=97.975
>epoch=8, lrate=0.800, error=97.146
>epoch=9, lrate=0.800, error=96.858
>epoch=10, lrate=0.800, error=96.802
>epoch=11, lrate=0.800, error=96.781
>epoch=12, lrate=0.800, error=96.770
>epoch=13, lrate=0.800, error=96.763
>epoch=14, lrate=0.800, error=96.758
>epoch=15, lrate=0.800, error=96.754
>epoch=16, lrate=0.800, error=96.751
>epoch=17, lrate=0.800, error=96.749
>epoch=18, lrate=0.800, error=96.747
>epoch=19, lrate=0.800, error=96.746
>epoch=20, lrate=0.800, error=96.744
>epoch=21, lrate=0.800, error=96.743
>epoch=22, lrate=0.800, error=96.742
>epoch=23, lrate=0.800, error=96.741
>epoch=24, lrate=0.800, error=96.741
>epoch=25, lrate=0.800, error=96.740
>epoch=26, lrate=0.800, error=96.739
>epoch=27, lrate=0.800, error=96.739
>epoch=28, lrate=0.800, error=96.738
>epoch=

>epoch=0, lrate=0.800, error=105.689
>epoch=1, lrate=0.800, error=103.807
>epoch=2, lrate=0.800, error=102.864
>epoch=3, lrate=0.800, error=102.575
>epoch=4, lrate=0.800, error=102.475
>epoch=5, lrate=0.800, error=102.356
>epoch=6, lrate=0.800, error=102.038
>epoch=7, lrate=0.800, error=101.053
>epoch=8, lrate=0.800, error=100.282
>epoch=9, lrate=0.800, error=100.135
>epoch=10, lrate=0.800, error=100.095
>epoch=11, lrate=0.800, error=100.077
>epoch=12, lrate=0.800, error=100.066
>epoch=13, lrate=0.800, error=100.059
>epoch=14, lrate=0.800, error=100.053
>epoch=15, lrate=0.800, error=100.049
>epoch=16, lrate=0.800, error=100.045
>epoch=17, lrate=0.800, error=100.041
>epoch=18, lrate=0.800, error=100.038
>epoch=19, lrate=0.800, error=100.035
>epoch=20, lrate=0.800, error=100.031
>epoch=21, lrate=0.800, error=100.028
>epoch=22, lrate=0.800, error=100.024
>epoch=23, lrate=0.800, error=100.019
>epoch=24, lrate=0.800, error=100.014
>epoch=25, lrate=0.800, error=100.008
>epoch=26, lrate=0.800

>epoch=3, lrate=0.800, error=104.179
>epoch=4, lrate=0.800, error=103.933
>epoch=5, lrate=0.800, error=103.115
>epoch=6, lrate=0.800, error=102.093
>epoch=7, lrate=0.800, error=101.204
>epoch=8, lrate=0.800, error=99.746
>epoch=9, lrate=0.800, error=98.370
>epoch=10, lrate=0.800, error=97.712
>epoch=11, lrate=0.800, error=97.568
>epoch=12, lrate=0.800, error=97.519
>epoch=13, lrate=0.800, error=97.494
>epoch=14, lrate=0.800, error=97.479
>epoch=15, lrate=0.800, error=97.469
>epoch=16, lrate=0.800, error=97.462
>epoch=17, lrate=0.800, error=97.457
>epoch=18, lrate=0.800, error=97.453
>epoch=19, lrate=0.800, error=97.449
>epoch=20, lrate=0.800, error=97.447
>epoch=21, lrate=0.800, error=97.444
>epoch=22, lrate=0.800, error=97.442
>epoch=23, lrate=0.800, error=97.441
>epoch=24, lrate=0.800, error=97.439
>epoch=25, lrate=0.800, error=97.438
>epoch=26, lrate=0.800, error=97.437
>epoch=27, lrate=0.800, error=97.436
>epoch=28, lrate=0.800, error=97.435
>epoch=29, lrate=0.800, error=97.434
>ep

...

>epoch=7, lrate=0.800, error=99.751
>epoch=8, lrate=0.800, error=99.735
>epoch=9, lrate=0.800, error=99.725
>epoch=10, lrate=0.800, error=99.717
>epoch=11, lrate=0.800, error=99.710
>epoch=12, lrate=0.800, error=99.704
>epoch=13, lrate=0.800, error=99.698
>epoch=14, lrate=0.800, error=99.693
>epoch=15, lrate=0.800, error=99.687
>epoch=16, lrate=0.800, error=99.679
>epoch=17, lrate=0.800, error=99.671
>epoch=18, lrate=0.800, error=99.659
>epoch=19, lrate=0.800, error=99.644
>epoch=20, lrate=0.800, error=99.620
>epoch=21, lrate=0.800, error=99.577
>epoch=22, lrate=0.800, error=99.484
>epoch=23, lrate=0.800, error=99.190
>epoch=24, lrate=0.800, error=98.239
>epoch=25, lrate=0.800, error=97.578
>epoch=26, lrate=0.800, error=97.454
>epoch=27, lrate=0.800, error=97.422
>epoch=28, lrate=0.800, error=97.407
>epoch=29, lrate=0.800, error=97.399
>epoch=30, lrate=0.800, error=97.394
>epoch=31, lrate=0.800, error=97.390
>epoch=32, lrate=0.800, error=97.388
>epoch=33, lrate=0.800, error=97.386
>epo

>epoch=0, lrate=0.800, error=104.147
>epoch=1, lrate=0.800, error=102.302
>epoch=2, lrate=0.800, error=101.942
>epoch=3, lrate=0.800, error=100.961
>epoch=4, lrate=0.800, error=99.813
>epoch=5, lrate=0.800, error=99.276
>epoch=6, lrate=0.800, error=99.155
>epoch=7, lrate=0.800, error=99.108
>epoch=8, lrate=0.800, error=99.072
>epoch=9, lrate=0.800, error=99.028
>epoch=10, lrate=0.800, error=98.949
>epoch=11, lrate=0.800, error=98.716
>epoch=12, lrate=0.800, error=97.968
>epoch=13, lrate=0.800, error=97.353
>epoch=14, lrate=0.800, error=97.228
>epoch=15, lrate=0.800, error=97.197
>epoch=16, lrate=0.800, error=97.183
>epoch=17, lrate=0.800, error=97.174
>epoch=18, lrate=0.800, error=97.169
>epoch=19, lrate=0.800, error=97.165
>epoch=20, lrate=0.800, error=97.163
>epoch=21, lrate=0.800, error=97.160
>epoch=22, lrate=0.800, error=97.159
>epoch=23, lrate=0.800, error=97.157
>epoch=24, lrate=0.800, error=97.156
>epoch=25, lrate=0.800, error=97.155
>epoch=26, lrate=0.800, error=97.154
>epoch=

>epoch=0, lrate=0.800, error=103.793
>epoch=1, lrate=0.800, error=101.951
>epoch=2, lrate=0.800, error=101.179
>epoch=3, lrate=0.800, error=100.931
>epoch=4, lrate=0.800, error=100.784
>epoch=5, lrate=0.800, error=100.516
>epoch=6, lrate=0.800, error=99.819
>epoch=7, lrate=0.800, error=98.726
>epoch=8, lrate=0.800, error=97.675
>epoch=9, lrate=0.800, error=97.167
>epoch=10, lrate=0.800, error=97.069
>epoch=11, lrate=0.800, error=97.036
>epoch=12, lrate=0.800, error=97.020
>epoch=13, lrate=0.800, error=97.009
>epoch=14, lrate=0.800, error=97.003
>epoch=15, lrate=0.800, error=96.998
>epoch=16, lrate=0.800, error=96.994
>epoch=17, lrate=0.800, error=96.991
>epoch=18, lrate=0.800, error=96.989
>epoch=19, lrate=0.800, error=96.987
>epoch=20, lrate=0.800, error=96.985
>epoch=21, lrate=0.800, error=96.984
>epoch=22, lrate=0.800, error=96.983
>epoch=23, lrate=0.800, error=96.982
>epoch=24, lrate=0.800, error=96.981
>epoch=25, lrate=0.800, error=96.980
>epoch=26, lrate=0.800, error=96.979
>epoc

>epoch=6, lrate=0.800, error=99.139
>epoch=7, lrate=0.800, error=99.117
>epoch=8, lrate=0.800, error=99.097
>epoch=9, lrate=0.800, error=99.074
>epoch=10, lrate=0.800, error=99.041
>epoch=11, lrate=0.800, error=98.982
>epoch=12, lrate=0.800, error=98.832
>epoch=13, lrate=0.800, error=98.311
>epoch=14, lrate=0.800, error=97.498
>epoch=15, lrate=0.800, error=97.238
>epoch=16, lrate=0.800, error=97.188
>epoch=17, lrate=0.800, error=97.169
>epoch=18, lrate=0.800, error=97.160
>epoch=19, lrate=0.800, error=97.154
>epoch=20, lrate=0.800, error=97.150
>epoch=21, lrate=0.800, error=97.147
>epoch=22, lrate=0.800, error=97.144
>epoch=23, lrate=0.800, error=97.143
>epoch=24, lrate=0.800, error=97.141
>epoch=25, lrate=0.800, error=97.140
>epoch=26, lrate=0.800, error=97.139
>epoch=27, lrate=0.800, error=97.138
>epoch=28, lrate=0.800, error=97.137
>epoch=29, lrate=0.800, error=97.136
>epoch=30, lrate=0.800, error=97.136
>epoch=31, lrate=0.800, error=97.135
>epoch=32, lrate=0.800, error=97.135
>epoc

>epoch=5, lrate=0.800, error=100.429
>epoch=6, lrate=0.800, error=99.507
>epoch=7, lrate=0.800, error=98.934
>epoch=8, lrate=0.800, error=98.781
>epoch=9, lrate=0.800, error=98.654
>epoch=10, lrate=0.800, error=98.339
>epoch=11, lrate=0.800, error=97.543
>epoch=12, lrate=0.800, error=97.019
>epoch=13, lrate=0.800, error=96.918
>epoch=14, lrate=0.800, error=96.889
>epoch=15, lrate=0.800, error=96.875
>epoch=16, lrate=0.800, error=96.867
>epoch=17, lrate=0.800, error=96.861
>epoch=18, lrate=0.800, error=96.857
>epoch=19, lrate=0.800, error=96.854
>epoch=20, lrate=0.800, error=96.852
>epoch=21, lrate=0.800, error=96.850
>epoch=22, lrate=0.800, error=96.848
>epoch=23, lrate=0.800, error=96.847
>epoch=24, lrate=0.800, error=96.846
>epoch=25, lrate=0.800, error=96.845
>epoch=26, lrate=0.800, error=96.844
>epoch=27, lrate=0.800, error=96.843
>epoch=28, lrate=0.800, error=96.842
>epoch=29, lrate=0.800, error=96.842
>epoch=30, lrate=0.800, error=96.841
>epoch=31, lrate=0.800, error=96.841
>epoc

In [21]:
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.09, 20, n_outputs)
#for layer in network:
    #print(layer)

for row in dataset:
    prediction = predict(network, row)
    #print('Expected=%d, Got=%d' % (row[-1], prediction))
    result = ['Expected=%d, Got=%d' % (row[-1], prediction)]
    #result.to_csv(os.path.join(dir,"test_result.csv"), index=False)
    print(result)

TypeError: 'int' object is not subscriptable

In [ ]:

#from datetime import datetime
import numpy as np
import pandas as pd
import csv
import os
import datetime
import time

from datetime import datetime
extension = ".csv"
date = datetime.now().strftime("%Y_%m_%d")
filename = date + extension
dir = "../result"
#x = datetime.date.today()
#date = datetime.now().strftime("%Y_%m_%d")
if not os.path.exists(dir):
    os.mkdir(dir)

viscous = 0.02  
radius = 0.01
A_YM = 300 *(10**6)
B_YM = 5 *(10**6)
ratio = round(A_YM/B_YM,3)
percentage = round(((A_YM-B_YM)/A_YM)*100,3)

    
accuracy = sum(scores)/float(len(scores))
result = [radius, date,viscous,  n_folds, n_epoch, l_rate, A_YM, B_YM, ratio,percentage, round(accuracy,3)]
with open(os.path.join(dir,filename),'a',newline ="") as f:
    writer = csv.writer(f)
    writer.writerow(["radius(%)","date", "viscous", "n_folds", "n_epoch", "l_rate","A_YM","B_YM","ratio","percentage","Accuracy%"])
    writer.writerow(result)
    
#open('myfile.%d.dat' % time.time(), 'w+')